# Prayer times manager
A Notebook for prayer-time management, Alhumdulillah

In [1]:
from IPython.display import HTML # For the last function below
import urllib.request as ur, pandas as pd, numpy as np, os
from html_table_parser import HTMLTableParser
import datetime

# For fetching the month's prayer-table (from KhaleejTimes)
def webtables(url, headers=''):
    req = ur.Request(url=url,headers={'User-Agent': headers})
    f = ur.urlopen(req)
    xhtml = f.read().decode('utf-8')
    p = HTMLTableParser()
    p.feed(xhtml)
    
    return p.tables


# For loading new/saved month's prayers-timing dataframe
def khaleej_prayers(name):
    # name = current month-name OR .npy file-name

    if name.endswith('.npy'): # if .npy file-name given, load this file.
        return np.load(name, allow_pickle=True).item()['df']
    
    # Else get (and save) a new prayers-table from KhaleejTimes:
    url = 'https://www.khaleejtimes.com/prayer-time-uae'
    uae_prayers_df = webtables(url, 'muslim-akhi')

    if len(uae_prayers_df) == 1:
        uae_prayers_df = uae_prayers_df[0]

    uae_prayers_df = pd.DataFrame(uae_prayers_df)
    uae_prayers_df.columns = uae_prayers_df.loc[0]
    uae_prayers_df = uae_prayers_df[1:].reset_index(drop=True)
    uae_prayers_df[name] = uae_prayers_df[name].astype(int)

    np.save(f'Islam/uae_prayers_{name}', 
            {'df':uae_prayers_df}, allow_pickle=True)
    
    return  uae_prayers_df


# For file management and dataframe-loading:
today = datetime.datetime.today()#.strftime('%d-%m-%Y')
month_now = today.strftime('%B')

def prayersDF():
    folder = 'Islam'
    prayers_file = [i for i in os.listdir(folder) if '_prayers_' in i]
    file_comment = f'New month - {month_now} Prayer-times loaded.%s'

    #prayers_file = []
    if prayers_file:
        file_path = folder+'/'+prayers_file[0]

        # If the file is NOT based on the current month's prayer times
        if not file_path.endswith(month_now+'.npy'):
            os.remove(file_path) # Remove the "previous month" file.
        else:
            print(file_comment.replace('New month','File')%'')
            return khaleej_prayers(file_path) # Load the saved file.
    
    print(file_comment%' (and saved)')
    return khaleej_prayers(month_now)


# Display DataFrames side-by-side (via HTML)
sideBYside_dfs = lambda dfs: HTML('<table><tr style="background-color:white;">'+
                    ''.join(['<td>'+table._repr_html_()+'</td>' for table in dfs])+
                                                '</tr></table>')

In [2]:
prayers_df = prayersDF() # Load our new/saved dataframe

today_info = prayers_df[prayers_df[month_now] == today.day]
today_prayers = today_info.T[4:].rename(columns={today_info.index[0]:'times'}).rename_axis('prayers')

print("Left: Prayers-DF (head)\t\t||\t"+
      f"Right: Today's prayer-times ({today.strftime('%d-%m-%Y')})")
display(sideBYside_dfs([prayers_df.head(), today_prayers]))

month_days_left = prayers_df.loc[today_info.index[0]:]

days_left = month_days_left.shape[0]-1
days_comment = '%d days left for '+month_now

if days_left:
    print(days_comment%days_left)
    display(month_days_left[1:])
else:
    print('Today is the last day '+days_comment.split()[-1][:-1]+'-day')

File - August Prayer-times loaded.
Left: Prayers-DF (head)		||	Right: Today's prayer-times (30-08-2022)


1 days left for August


,August,Day,Hijri,Month,Fajr,Sunrise,Dhuhr,Asr,Maghrib,Isha
30,31,Wednesday,4,Safar,4:40 AM,5:56 AM,12:22 PM,3:49 PM,6:42 PM,7:58 PM


### Iqamah timings (from local masjid)

In [3]:
iqamah = {i:pd.Timedelta(minutes=j) if j else np.nan 
          for i,j in zip(today_prayers.index, [20,0,15,15,2,15])}

print('Iqamah times (my local masjid):')
display(iqamah)

prayers_astime = pd.to_datetime(today_prayers.times, format='%I:%M %p')

today_prayers['iqamah'] = prayers_astime.values + today_prayers.index.map(iqamah.get).values
today_prayers['iqamah'] = today_prayers['iqamah'].dt.strftime('%I:%M %p')
today_prayers

Iqamah times (my local masjid):


{'Fajr': Timedelta('0 days 00:20:00'),
 'Sunrise': nan,
 'Dhuhr': Timedelta('0 days 00:15:00'),
 'Asr': Timedelta('0 days 00:15:00'),
 'Maghrib': Timedelta('0 days 00:02:00'),
 'Isha': Timedelta('0 days 00:15:00')}

,times,iqamah
prayers,,
Fajr,4:39 AM,04:59 AM
Sunrise,5:55 AM,NaN
Dhuhr,12:22 PM,12:37 PM
Asr,3:50 PM,04:05 PM
Maghrib,6:43 PM,06:45 PM
Isha,7:59 PM,08:14 PM


### Add durations until the next prayer/"session"
For each row in *today_prayers*, we will find the difference in time-duration or "time until next prayer" per row. 

<u>Isha's *until_next*</u> below is the duration from **today's Isha until today's Fajr** - assuming the same Fajr-time for the next day.

In [4]:
today_prayers['until_next'] = prayers_astime.diff().fillna(pd.Timedelta(seconds=0))
#Or: diff().dt.seconds.fillna(0).apply(duration, 0) # duration() from my repo: Python-Daily/Date & Time/duration

today_prayers.loc['Fajr', 'until_next'] = prayers_astime.iloc[::-5].diff()[-1]
today_prayers['until_next'] = today_prayers.until_next.apply(lambda x: str(x).split()[-1])
today_prayers['until_next'] = np.append(today_prayers.until_next[1:].values, 
                                        today_prayers.loc['Fajr', 'until_next'])
today_prayers

,times,iqamah,until_next
prayers,,,
Fajr,4:39 AM,04:59 AM,01:16:00
Sunrise,5:55 AM,NaN,06:27:00
Dhuhr,12:22 PM,12:37 PM,03:28:00
Asr,3:50 PM,04:05 PM,02:53:00
Maghrib,6:43 PM,06:45 PM,01:16:00
Isha,7:59 PM,08:14 PM,+08:40:00


 ### Next to be looked at is inputting in end-times per prayer, to be continued InshaAllah